<a href="https://colab.research.google.com/github/agonzalezl2025/Parcial2/blob/main/Parcial2HE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**0. Preparación del entorno**

## 0.1: Carga de datos

In [20]:
# === 1. Carga de Datos y Preprocesamiento ===

# Instalación silenciosa de librerías

# Frameworks de Deep Learning
!pip install -q tensorflow torch keras keras-tuner

# Análisis y manipulación de datos
!pip install -q pandas numpy scikit-learn

# Visualización
!pip install -q matplotlib seaborn plotly

# Optimización y evaluación
!pip install -q optuna tensorboard scikit-optimize

# Utilidades
!pip install -q tqdm joblib

# Análisis exploratorio y acceso a datos
!pip install -q ydata-profiling datasets huggingface_hub kaggle

# Importación de librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from ydata_profiling import ProfileReport


## 0.2: Definición de semilla

In [19]:
# Definir la semilla

SEED = 42

# **1. Carga y unión de los datasets**

## 1.1: Descargar y cargar los datasets desde Kaggle

In [3]:
# === 1.1 Descargar y Cargar los Datasets desde Kaggle ===
!kaggle datasets download -d sazidthe1/world-gdp-data
!unzip world-gdp-data.zip

# Cargar los datasets con la ruta correcta
gdp_data = pd.read_csv("gdp_data.csv")
country_codes = pd.read_csv("country_codes.csv")



Dataset URL: https://www.kaggle.com/datasets/sazidthe1/world-gdp-data
License(s): Attribution 4.0 International (CC BY 4.0)
  0% 0.00/108k [00:00<?, ?B/s]
100% 108k/108k [00:00<00:00, 50.4MB/s]
Archive:  world-gdp-data.zip
  inflating: country_codes.csv       
  inflating: gdp_data.csv            


##  1.2: Inspección de los datos

In [4]:
# === 1.2 Inspección de los datos ===
print("Primeras filas de GDP Data:")
print(gdp_data.head())

print("\nPrimeras filas de Country Codes:")
print(country_codes.head())

Primeras filas de GDP Data:
  country_name country_code  year        value
0  Afghanistan          AFG  1960  537777811.1
1  Afghanistan          AFG  1961  548888895.6
2  Afghanistan          AFG  1962  546666677.8
3  Afghanistan          AFG  1963  751111191.1
4  Afghanistan          AFG  1964  800000044.4

Primeras filas de Country Codes:
  country_code                     region         income_group
0          ABW  Latin America & Caribbean          High income
1          AFG                 South Asia           Low income
2          AGO         Sub-Saharan Africa  Lower middle income
3          ALB      Europe & Central Asia  Upper middle income
4          AND      Europe & Central Asia          High income


## 1.3: Unir datasets

In [10]:
# === 1.3 Unir datasets ===
df = pd.merge(gdp_data, country_codes, on='country_code', how='inner')

df.head()


,country_name,country_code,year,value,region,income_group
0,Afghanistan,AFG,1960,537777811.1,South Asia,Low income
1,Afghanistan,AFG,1961,548888895.6,South Asia,Low income
2,Afghanistan,AFG,1962,546666677.8,South Asia,Low income
3,Afghanistan,AFG,1963,751111191.1,South Asia,Low income
4,Afghanistan,AFG,1964,800000044.4,South Asia,Low income


# **2. Clasificación y separación de datos train/test**

##2.1: Clasificación de datos en terciles

In [16]:
import numpy as np
import pandas as pd

# === Función para calcular terciles por año ===
def categorize_gdp_by_year(row, p30_dict, p70_dict):
    year = row['year']
    value = row['value']

    # Tomamos los terciles correspondientes a ese año
    p30 = p30_dict.get(year, np.nan)
    p70 = p70_dict.get(year, np.nan)

    # Clasificación en 30%-40%-30%
    if value < p30:
        return 'Low'
    elif value < p70:
        return 'Medium'
    else:
        return 'High'

# Calcular terciles (30% y 70%) para cada año
p30_by_year = df.groupby('year')['value'].quantile(0.30).to_dict()
p70_by_year = df.groupby('year')['value'].quantile(0.70).to_dict()

# Aplicar la categorización por año usando los terciles
df['historic_gdp'] = df.apply(lambda row: categorize_gdp_by_year(row, p30_by_year, p70_by_year), axis=1)


# Verificar la distribución de categorías después del encoding
print(df.head())

        country_name country_code  year         value  \
0        Afghanistan          AFG  1960  5.377778e+08   
6938       Nicaragua          NIC  1960  2.272233e+08   
10422         Zambia          ZMB  1960  7.130000e+08   
7385          Panama          PAN  1960  5.371471e+08   
10252  Venezuela, RB          VEN  1960  7.663938e+09   

                          region         income_group gdp_category  \
0                     South Asia           Low income       Medium   
6938   Latin America & Caribbean  Lower middle income          Low   
10422         Sub-Saharan Africa  Lower middle income       Medium   
7385   Latin America & Caribbean          High income       Medium   
10252  Latin America & Caribbean  Upper middle income         High   

      historic_gdp_category historic_gdp  
0                    Medium       Medium  
6938                    Low          Low  
10422                Medium       Medium  
7385                 Medium       Medium  
10252                

## 2.2 Separaración entre 80% train y 20% test

In [17]:

# Ordenar los datos por 'year' para respetar la serie temporal
df = df.sort_values(by='year')

# Determinar el año de corte para el 80% de los datos
year_cutoff = int(round(df['year'].quantile(0.80)))

print(f"Año de corte para Train-Test: {year_cutoff}")

# Dividir en conjunto de entrenamiento (antes o igual al año de corte) y prueba (después del año de corte)
df_train = df[df['year'] <= year_cutoff].copy()
df_test = df[df['year'] > year_cutoff].copy()

# Mostrar los primeros datos de entrenamiento y prueba
print("\nPrimeros datos de entrenamiento:")
display(df_train.head())

print("\nPrimeros datos de prueba:")
display(df_test.head())

Año de corte para Train-Test: 2012

Primeros datos de entrenamiento:


,country_name,country_code,year,value,region,income_group,gdp_category,historic_gdp_category,historic_gdp
0,Afghanistan,AFG,1960,5.377778e+08,South Asia,Low income,Medium,Medium,Medium
8828,St. Kitts and Nevis,KNA,1960,1.236664e+07,Latin America & Caribbean,High income,Low,Low,Low
2083,Colombia,COL,1960,4.031153e+09,Latin America & Caribbean,Upper middle income,High,High,High
8399,Singapore,SGP,1960,7.047517e+08,East Asia & Pacific,High income,Medium,Medium,Medium
909,Belize,BLZ,1960,2.807248e+07,Latin America & Caribbean,Upper middle income,Low,Low,Low



Primeros datos de prueba:


,country_name,country_code,year,value,region,income_group,gdp_category,historic_gdp_category,historic_gdp
2179,Comoros,COM,2013,1.116224e+09,Sub-Saharan Africa,Lower middle income,Low,Low,Low
1406,Brunei Darussalam,BRN,2013,1.809433e+10,East Asia & Pacific,High income,Medium,Medium,Medium
2242,"Congo, Dem. Rep.",COD,2013,3.267975e+10,Sub-Saharan Africa,Low income,Medium,Medium,Medium
10538,Zimbabwe,ZWE,2013,1.909102e+10,Sub-Saharan Africa,Lower middle income,Medium,Medium,Medium
9423,Thailand,THA,2013,4.200000e+11,East Asia & Pacific,Upper middle income,High,High,High


# **Análisis de datos**

## Diccionario

In [ ]:
dict_by_country = (
    df
    .groupby('country_code')
    .apply(lambda x: x.to_dict(orient='records'))
    .to_dict()
)

# Ejemplo: mostrar el contenido para 'AFG'
dict_by_country['BEN']

## Análisis de datos: TRAIN

In [8]:
#Confirmamos los datos TRAIN:
df_train.head()

In [ ]:
#Análisis descriptivo de los datos de entrenamiento sin nigun cambio previo:
from ydata_profiling import ProfileReport
reporte_train = ProfileReport(df_train, title="Profiling Report Train dataset")
reporte_train.to_file("reporte_train.html")
reporte_train

## Análisis de datos: TEST

In [8]:
#Confirmamos los datos TEST:
df_test.head()

In [ ]:
#Análisis descriptivo de los datos de entrenamiento sin nigun cambio previo:
from ydata_profiling import ProfileReport
reporte_test = ProfileReport(df_test, title="Profiling Report Test dataset")
reporte_test.to_file("reporte_test.html")
reporte_test

# Separar variables X y Y

## Pasar de variables categóricas a binarias

In [ ]:
import pandas as pd

# === 1. Asegurar que 'gdp_category' no esté en formato binario antes de procesar ===
if 'GDP_Low' in df.columns or 'GDP_Medium' in df.columns or 'GDP_High' in df.columns:
    df = df.drop(columns=['GDP_Low', 'GDP_Medium', 'GDP_High'], errors='ignore')

# === 2. Aplicar One-Hot Encoding a la columna 'gdp_category' ===
df = pd.get_dummies(df, columns=['gdp_category'], prefix='GDP')

## Separar

In [ ]:

# === 3. Separar en conjuntos de entrenamiento y prueba con el 80%-20% basado en el tiempo ===
df = df.sort_values(by='year')  # Ordenar los datos cronológicamente
year_cutoff = int(round(df['year'].quantile(0.80)))  # Obtener el año de corte para 80%

# División en entrenamiento y prueba
df_train = df[df['year'] <= year_cutoff].copy()
df_test = df[df['year'] > year_cutoff].copy()

# === 4. Separar en X (features) e y (labels) ===
X_train = df_train.drop(columns=['GDP_Low', 'GDP_Medium', 'GDP_High'])  # Variables independientes
y_train = df_train[['GDP_Low', 'GDP_Medium', 'GDP_High']]  # Variables dependientes binarias

X_test = df_test.drop(columns=['GDP_Low', 'GDP_Medium', 'GDP_High'])
y_test = df_test[['GDP_Low', 'GDP_Medium', 'GDP_High']]

# === 5. Verificar resultados ===
print(f"Tamaño de X_train: {X_train.shape}, Tamaño de y_train: {y_train.shape}")
print(f"Tamaño de X_test: {X_test.shape}, Tamaño de y_test: {y_test.shape}")

# Mostrar la distribución de los años en cada conjunto
print("\nAños en entrenamiento:", df_train['year'].unique())
print("Años en prueba:", df_test['year'].unique())

# Mostrar las primeras filas de X e y
display(X_train.head(), y_train.head())
display(X_test.head(), y_test.head())